# Pydantic AI Walkthrough

Inspired bij: https://ai.pydantic.dev/agents/

Setup the environment

In [ ]:
import dotenv
import os
import logfire

import tool_functions as tf

import nest_asyncio

nest_asyncio.apply()

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")
logfire_token = os.getenv("LOGFIRE_TOKEN")

# configure logfire
logfire.configure(token=logfire_token)
logfire.instrument_pydantic_ai()

Logfire project URL: ]8;id=123714;https://logfire-eu.pydantic.dev/elbrink/claude-test\https://logfire-eu.pydantic.dev/elbrink/claude-test]8;;\

1 Definieer een basis agent

In [3]:
from pydantic_ai import Agent
from pydantic_ai.models.anthropic import AnthropicModel

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = AnthropicModel(os.getenv("DEFAULT_MODEL"))

basic_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)

Stel een vraag aan de agent

In [4]:
result = basic_agent.run_sync("geef een defintie van een agent")
print(result.output)

15:33:32.063 basic_agent run
15:33:32.064   chat claude-3-haiku-20240307
Een agent is een autonoom systeem dat zich in een bepaalde omgeving bevindt en daar acties in onderneemt om bepaalde doelen te bereiken. De belangrijkste kenmerken van een agent zijn:

1. Autonomie: De agent kan zelfstandig beslissingen nemen en acties uitvoeren zonder voortdurende menselijke tussenkomst.

2. Waarneming: De agent kan informatie waarnemen over zijn omgeving via sensoren, zoals camera's, microfoons of andere invoerkanalen.

3. Doelgerichtheid: De agent heeft specifieke doelen of taken die hij probeert te bereiken door middel van zijn acties.

4. Proactief handelen: De agent kan zelf initiatieven nemen en pro-actief handelen om zijn doelen te bereiken, in plaats van alleen reactief te reageren op externe prikkels.

5. Flexibiliteit: Een agent kan zich aanpassen aan veranderende omstandigheden en nieuwe situaties, en zijn gedrag en besluitvorming daarop afstemmen.

Agenten kunnen in vele vormen voorko

Definieer tools voor de agent

In [5]:
tool_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)


@tool_agent.tool_plain
def tool_bereken_btw(bedrag: float, btw_percentage: float):
    """
    Bereken BTW over een bedrag.
    """
    return tf.calculate_btw(bedrag, btw_percentage)


@tool_agent.tool_plain
def tool_bereken_korting(bedrag: float, korting_percentage: float):
    """
    Bereken korting over een bedrag.
    """
    return tf.calculate_discount(bedrag, korting_percentage)


@tool_agent.tool_plain
def tool_get_inwoners_gemeente(gemeente_naam: str, jaar: str):
    """
    Haal inwonersaantallen op van Nederlandse gemeenten via CBS API.
    """
    return tf.get_inwoners_gemeente(gemeente_naam, jaar)

Test een aantal tool calls

In [6]:
result2 = tool_agent.run_sync(
    "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld"
)
print(result2.output)

15:33:42.048 tool_agent run
15:33:42.049   chat claude-3-haiku-20240307
15:33:43.569   running 1 tool
15:33:43.570     running tool: tool_bereken_btw
15:33:43.571   chat claude-3-haiku-20240307
15:33:45.028   running 1 tool
15:33:45.029     running tool: tool_bereken_korting
15:33:45.030   chat claude-3-haiku-20240307
Na de 15% korting op het bedrag inclusief BTW, komt de uiteindelijke prijs die je moet betalen neer op 102,85 euro.

Samengevat:
- Origineel bedrag exclusief BTW: 100 euro
- BTW (21%): 21 euro
- Totaalbedrag inclusief BTW: 121 euro
- Korting (15%): 18,15 euro
- Eindprijs na korting: 102,85 euro

Het overzicht is:
- Origineel bedrag excl. BTW: 100 euro
- BTW (21%): 21 euro
- Totaalbedrag incl. BTW: 121 euro
- Korting (15%): 18,15 euro
- Eindprijs na korting: 102,85 euro


In [7]:
result = tool_agent.run_sync(
    "Geef een overzicht van de top-7 gemeenten in Nederland met het grootste aantal inwoners in 2024. Laat ook het aantal inwoners van Groningen zien."
)
print(result.output)

15:33:56.284 tool_agent run
15:33:56.285   chat claude-3-haiku-20240307
15:34:00.696   running 7 tools
15:34:00.697     running tool: tool_get_inwoners_gemeente
15:34:00.697     running tool: tool_get_inwoners_gemeente
15:34:00.697     running tool: tool_get_inwoners_gemeente
15:34:00.698     running tool: tool_get_inwoners_gemeente
15:34:00.698     running tool: tool_get_inwoners_gemeente
15:34:00.698     running tool: tool_get_inwoners_gemeente
15:34:00.698     running tool: tool_get_inwoners_gemeente
15:34:02.035   chat claude-3-haiku-20240307
Op basis van de resultaten, hier de top-7 gemeenten met het grootste aantal inwoners in 2024:

1. Amsterdam: 931.298 inwoners
2. Rotterdam: 670.610 inwoners 
3. Utrecht: 374.238 inwoners
4. Eindhoven: 246.417 inwoners
5. Tilburg: 229.836 inwoners
6. Groningen: 243.768 inwoners
7. Breda: 188.078 inwoners

Dus de gemeente Groningen had in 2024 243.768 inwoners.


In [8]:
result = tool_agent.run_sync(
    "Als alle inwoners uit Eindhoven in 2025 15 euro betalen, hoeveel krijg ik dan."
)
print(result.output)

15:34:18.259 tool_agent run
15:34:18.260   chat claude-3-haiku-20240307
15:34:20.044   running 1 tool
15:34:20.044     running tool: tool_get_inwoners_gemeente
15:34:20.966   chat claude-3-haiku-20240307
Volgens de gegevens van het CBS zal Eindhoven in 2025 ongeveer 249.054 inwoners hebben.

Als elk van deze inwoners 15 euro betaalt, dan is de totale opbrengst:

249.054 inwoners x 15 euro = 3.735.810 euro

Dus als alle inwoners van Eindhoven in 2025 elk 15 euro betalen, krijg je in totaal 3.735.810 euro.


Voeg geheugen toe aan de agent

In [9]:
mem_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)

result = mem_agent.run_sync(
    "wat was ook alweer het btw bedrag van het eerder gekochte product X en hoeveel korting kreeg ik. Zoek dit uit de conversatie hiervoor.",
    message_history=result2.all_messages(),
)
print(result.output)

15:34:28.528 mem_agent run
15:34:28.528   chat claude-3-haiku-20240307
Goed, laten we dat even terugzoeken in de vorige stappen:

Uit de eerste stap blijkt:
- Het bedrag exclusief BTW was 100 euro
- Het BTW bedrag was 21 euro

Uit de tweede stap blijkt:
- De korting was 15% op het bedrag inclusief BTW van 121 euro
- Het kortingsbedrag was 18,15 euro

Dus samengevat:
- BTW bedrag: 21 euro
- Korting: 18,15 euro
